# 6-7. 프로젝트: 멋진 작사가 만들기

---

## Step 1. 데이터 다운로드

In [1]:
!wget https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip
!unzip song_lyrics.zip -d ~/aiffel/lyricist/data/lyrics  # lyrics 폴더에 압축풀기

--2021-01-26 11:05:30--  https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip
Resolving aiffelstaticprd.blob.core.windows.net (aiffelstaticprd.blob.core.windows.net)... 52.239.148.4
Connecting to aiffelstaticprd.blob.core.windows.net (aiffelstaticprd.blob.core.windows.net)|52.239.148.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2101791 (2.0M) [application/zip]
Saving to: ‘song_lyrics.zip’

song_lyrics.zip     100%[===================>]   2.00M  1.54MB/s    in 1.3s    

2021-01-26 11:05:31 (1.54 MB/s) - ‘song_lyrics.zip’ saved [2101791/2101791]

Archive:  song_lyrics.zip
  inflating: /home/aiffel-dj24/aiffel/lyricist/data/lyrics/Kanye_West.txt  
  inflating: /home/aiffel-dj24/aiffel/lyricist/data/lyrics/Lil_Wayne.txt  
  inflating: /home/aiffel-dj24/aiffel/lyricist/data/lyrics/adele.txt  
  inflating: /home/aiffel-dj24/aiffel/lyricist/data/lyrics/al-green.txt  
  inflating: /home/aiffel-dj24/aiffel/lyricist/data/lyrics/alicia-keys.

---

## Step 2. 데이터 읽어오기

In [1]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Load up on guns, bring your friends', "It's fun to lose and to pretend", "She's over-bored and self-assured"]


---

## Step 3. 데이터 정제

### 공백 문장 제거

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    #if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Load up on guns, bring your friends
It's fun to lose and to pretend
She's over-bored and self-assured
Oh no, I know a dirty word Hello, hello, hello, how low
Hello, hello, hello, how low
Hello, hello, hello, how low
Hello, hello, hello With the lights out, it's less dangerous
Here we are now, entertain us
I feel stupid and contagious
Here we are now, entertain us


### 토큰화

In [3]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!
import os

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


### 정제 데이터 구축

In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> load up on guns , bring your friends <end>',
 '<start> it s fun to lose and to pretend <end>',
 '<start> she s over bored and self assured <end>',
 '<start> oh no , i know a dirty word hello , hello , hello , how low <end>',
 '<start> hello , hello , hello , how low <end>',
 '<start> hello , hello , hello , how low <end>',
 '<start> hello , hello , hello with the lights out , it s less dangerous <end>',
 '<start> here we are now , entertain us <end>',
 '<start> i feel stupid and contagious <end>',
 '<start> here we are now , entertain us <end>']

### 데이터를 숫자로 변환

In [5]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=15000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.
    tensor_15 = []
    for voca in tensor:
        if len(voca) < 16:    # 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외하도록 설정
            tensor_15.append(voca)
            
    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor_15, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2 2347   29 ...    0    0    0]
 [   2   11   16 ...    0    0    0]
 [   2   48   16 ...    0    0    0]
 ...
 [   2  135   50 ...    0    0    0]
 [   2  135    4 ...    0    0    0]
 [   2  135    4 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fdd9fe22fd0>


In [6]:
print(tensor[:3, :15])

[[   2 2347   29   18 1100    4  307   21  262    3    0    0    0    0
     0]
 [   2   11   16  531   10  384    8   10 1019    3    0    0    0    0
     0]
 [   2   48   16  146 2963    8 1202 6481    3    0    0    0    0    0
     0]]


In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [8]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[   2 2347   29   18 1100    4  307   21  262    3    0    0    0    0]
[2347   29   18 1100    4  307   21  262    3    0    0    0    0    0]


---

## Step 4. 평가 데이터셋 분리

In [9]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input,test_size=0.2,shuffle=True)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)
print("Source Train:", enc_val.shape)
print("Target Train:", dec_val.shape)

Source Train: (124810, 14)
Target Train: (124810, 14)
Source Train: (31203, 14)
Target Train: (31203, 14)


In [10]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

In [11]:
train_dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)
train_dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [12]:
val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)
val_dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

---

---

## Step 5. 인공지능 만들기

### embedding_size = 256 / hidden_size = 1024

In [105]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [106]:
for src_sample, tgt_sample in train_dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 15001), dtype=float32, numpy=
array([[[ 1.20993071e-04, -9.35731896e-06, -5.16455002e-05, ...,
          5.54307953e-05, -6.00745552e-05,  2.16923654e-04],
        [ 1.39268755e-04,  4.22480916e-05, -1.44037287e-04, ...,
          3.05972848e-04, -1.55333910e-04,  1.47773084e-04],
        [ 3.79328907e-04,  7.70818369e-05, -3.76106793e-04, ...,
          3.03419831e-04, -2.75298604e-04,  3.62912215e-05],
        ...,
        [ 4.54056681e-05,  5.91771328e-04, -3.93880706e-04, ...,
         -3.76343756e-04,  1.18543336e-04, -2.89995281e-04],
        [ 9.73325004e-05,  6.12279458e-04, -6.07421680e-04, ...,
         -6.53054216e-04, -3.22883978e-04, -6.21841871e-04],
        [ 2.40462483e-04,  5.73745230e-04, -8.71937722e-04, ...,
         -9.76615702e-04, -7.72480620e-04, -1.00003695e-03]],

       [[ 1.20993071e-04, -9.35731896e-06, -5.16455002e-05, ...,
          5.54307953e-05, -6.00745552e-05,  2.16923654e-04],
        [ 2.65595008e-04, -1.35764436e-04,  1

In [107]:
model.summary()

Model: "text_generator_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      multiple                  3840256   
_________________________________________________________________
lstm_12 (LSTM)               multiple                  5246976   
_________________________________________________________________
lstm_13 (LSTM)               multiple                  8392704   
_________________________________________________________________
dense_6 (Dense)              multiple                  15376025  
Total params: 32,855,961
Trainable params: 32,855,961
Non-trainable params: 0
_________________________________________________________________


In [108]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(train_dataset, epochs=10, validation_data=val_dataset)

Epoch 1/10
487/487 [==============================] - 74s 152ms/step - loss: 3.5664 - val_loss: 3.1793
Epoch 2/10
487/487 [==============================] - 75s 153ms/step - loss: 3.0487 - val_loss: 2.9977
Epoch 3/10
487/487 [==============================] - 75s 154ms/step - loss: 2.8897 - val_loss: 2.8915
Epoch 4/10
487/487 [==============================] - 77s 158ms/step - loss: 2.7689 - val_loss: 2.8146
Epoch 5/10
487/487 [==============================] - 76s 156ms/step - loss: 2.6656 - val_loss: 2.7562
Epoch 6/10
487/487 [==============================] - 75s 154ms/step - loss: 2.5723 - val_loss: 2.7032
Epoch 7/10
487/487 [==============================] - 75s 154ms/step - loss: 2.4847 - val_loss: 2.6606
Epoch 8/10
487/487 [==============================] - 76s 155ms/step - loss: 2.4028 - val_loss: 2.6270
Epoch 9/10
487/487 [==============================] - 76s 157ms/step - loss: 2.3263 - val_loss: 2.5977
Epoch 10/10
487/487 [==============================] - 75s 154ms/step - l

In [109]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
																 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [110]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m a liability <end> '

### embedding_size = 512 / hidden_size = 1024

In [111]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [112]:
for src_sample, tgt_sample in train_dataset.take(1): break
model(src_sample)
model.summary()

Model: "text_generator_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      multiple                  7680512   
_________________________________________________________________
lstm_14 (LSTM)               multiple                  6295552   
_________________________________________________________________
lstm_15 (LSTM)               multiple                  8392704   
_________________________________________________________________
dense_7 (Dense)              multiple                  15376025  
Total params: 37,744,793
Trainable params: 37,744,793
Non-trainable params: 0
_________________________________________________________________


In [113]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(train_dataset, epochs=10, validation_data=val_dataset)

Epoch 1/10
487/487 [==============================] - 85s 174ms/step - loss: 3.4809 - val_loss: 3.1359
Epoch 2/10
487/487 [==============================] - 85s 174ms/step - loss: 3.0100 - val_loss: 2.9567
Epoch 3/10
487/487 [==============================] - 85s 174ms/step - loss: 2.8414 - val_loss: 2.8471
Epoch 4/10
487/487 [==============================] - 85s 175ms/step - loss: 2.7120 - val_loss: 2.7664
Epoch 5/10
487/487 [==============================] - 85s 175ms/step - loss: 2.6005 - val_loss: 2.7058
Epoch 6/10
487/487 [==============================] - 85s 175ms/step - loss: 2.5002 - val_loss: 2.6579
Epoch 7/10
487/487 [==============================] - 85s 174ms/step - loss: 2.4078 - val_loss: 2.6166
Epoch 8/10
487/487 [==============================] - 85s 174ms/step - loss: 2.3221 - val_loss: 2.5794
Epoch 9/10
487/487 [==============================] - 85s 175ms/step - loss: 2.2414 - val_loss: 2.5494
Epoch 10/10
487/487 [==============================] - 86s 177ms/step - l

In [114]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
																 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [115]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

### embedding_size = 128 / hidden_size = 1024

In [119]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 128
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [120]:
for src_sample, tgt_sample in train_dataset.take(1): break
model(src_sample)
model.summary()

Model: "text_generator_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     multiple                  1920128   
_________________________________________________________________
lstm_20 (LSTM)               multiple                  4722688   
_________________________________________________________________
lstm_21 (LSTM)               multiple                  8392704   
_________________________________________________________________
dense_10 (Dense)             multiple                  15376025  
Total params: 30,411,545
Trainable params: 30,411,545
Non-trainable params: 0
_________________________________________________________________


In [121]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(train_dataset, epochs=10, validation_data=val_dataset)

Epoch 1/10
487/487 [==============================] - 70s 144ms/step - loss: 3.5661 - val_loss: 3.2329
Epoch 2/10
487/487 [==============================] - 71s 145ms/step - loss: 3.0964 - val_loss: 3.0298
Epoch 3/10
487/487 [==============================] - 71s 146ms/step - loss: 2.9290 - val_loss: 2.9282
Epoch 4/10
487/487 [==============================] - 71s 146ms/step - loss: 2.8117 - val_loss: 2.8522
Epoch 5/10
487/487 [==============================] - 74s 153ms/step - loss: 2.7133 - val_loss: 2.7949
Epoch 6/10
487/487 [==============================] - 72s 147ms/step - loss: 2.6260 - val_loss: 2.7452
Epoch 7/10
487/487 [==============================] - 71s 146ms/step - loss: 2.5460 - val_loss: 2.7077
Epoch 8/10
487/487 [==============================] - 71s 146ms/step - loss: 2.4714 - val_loss: 2.6760
Epoch 9/10
487/487 [==============================] - 73s 149ms/step - loss: 2.4009 - val_loss: 2.6435
Epoch 10/10
487/487 [==============================] - 72s 147ms/step - l

In [122]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
																 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [123]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you so much <end> '

### embedding_size = 256 / hidden_size = 2048

In [71]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [73]:
for src_sample, tgt_sample in train_dataset.take(1): break
model(src_sample)
model.summary()

Model: "text_generator_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      multiple                  3840256   
_________________________________________________________________
lstm_8 (LSTM)                multiple                  18882560  
_________________________________________________________________
lstm_9 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense_4 (Dense)              multiple                  30737049  
Total params: 87,022,489
Trainable params: 87,022,489
Non-trainable params: 0
_________________________________________________________________


In [74]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(train_dataset, epochs=10, validation_data=val_dataset)

Epoch 1/10
487/487 [==============================] - 208s 427ms/step - loss: 3.5155 - val_loss: 3.1006
Epoch 2/10
487/487 [==============================] - 211s 434ms/step - loss: 2.9830 - val_loss: 2.8991
Epoch 3/10
487/487 [==============================] - 210s 432ms/step - loss: 2.7694 - val_loss: 2.7554
Epoch 4/10
487/487 [==============================] - 214s 440ms/step - loss: 2.5848 - val_loss: 2.6515
Epoch 5/10
487/487 [==============================] - 211s 433ms/step - loss: 2.4156 - val_loss: 2.5692
Epoch 6/10
487/487 [==============================] - 213s 437ms/step - loss: 2.2576 - val_loss: 2.5021
Epoch 7/10
487/487 [==============================] - 217s 445ms/step - loss: 2.1079 - val_loss: 2.4468
Epoch 8/10
487/487 [==============================] - 212s 435ms/step - loss: 1.9652 - val_loss: 2.3973
Epoch 9/10
487/487 [==============================] - 211s 433ms/step - loss: 1.8287 - val_loss: 2.3604
Epoch 10/10
487/487 [==============================] - 214s 439m

In [ ]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
																 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [86]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love frighten frighten sew fitted fitted memphis memphis tyiest tyiest tyiest pastor golden mighty vamos vamos claustrophobia claustrophobia '

### embedding_size = 512 / hidden_size = 2048

In [13]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [14]:
for src_sample, tgt_sample in train_dataset.take(1): break
model(src_sample)
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  7680512   
_________________________________________________________________
lstm (LSTM)                  multiple                  20979712  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense (Dense)                multiple                  30737049  
Total params: 92,959,897
Trainable params: 92,959,897
Non-trainable params: 0
_________________________________________________________________


In [15]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(train_dataset, epochs=10, validation_data=val_dataset)

Epoch 1/10
487/487 [==============================] - 216s 444ms/step - loss: 3.3634 - val_loss: 2.9506
Epoch 2/10
487/487 [==============================] - 218s 448ms/step - loss: 2.7746 - val_loss: 2.6960
Epoch 3/10
487/487 [==============================] - 219s 450ms/step - loss: 2.4438 - val_loss: 2.5025
Epoch 4/10
487/487 [==============================] - 220s 452ms/step - loss: 2.1140 - val_loss: 2.3609
Epoch 5/10
487/487 [==============================] - 220s 451ms/step - loss: 1.8116 - val_loss: 2.2592
Epoch 6/10
487/487 [==============================] - 220s 452ms/step - loss: 1.5539 - val_loss: 2.1976
Epoch 7/10
487/487 [==============================] - 220s 451ms/step - loss: 1.3469 - val_loss: 2.1713
Epoch 8/10
487/487 [==============================] - 219s 450ms/step - loss: 1.1914 - val_loss: 2.1672
Epoch 9/10
487/487 [==============================] - 219s 451ms/step - loss: 1.0903 - val_loss: 2.1840
Epoch 10/10
487/487 [==============================] - 221s 453m

In [16]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
																 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [17]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m not gonna crack <end> '

---

## 루브릭 평가

__1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?__  
_(텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?)_

- 가사 텍스트 생성 모델이 정상적으로 동작하고 텍스트 제너레이션 결과 정상적인 문장이 작성되었다.

__2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?__  
_(특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가?)_

- 특수문자 제거, 토크나이저 생성, 패딩처리 등 과정이 빠짐없이 진행되었다.

__3. 텍스트 생성모델이 안정적으로 학습되었는가?__  
_(텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?)__

- embedding_size = 512, hidden_size = 2048로 진행을 했을 경우 validation loss를 2.2까지 낮출 수 있었다.

---

## 회고

__1. 프로젝트 진행시 어려웠던 점__

- 문장을 토큰화 했을 때 토크의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외하기 위한 코드를 수정한 점
- validation loss를 2.2 이하로 낮추기 위해 하이퍼파라미터를 높게 설정해 모델을 학습 하다보니 많은 시간이 소요된 점

__2. 프로젝트를 진행하면서 알아낸 점 또는 아직 모호한 점__

- 알아낸 점 :
    - 하이퍼파라미터 조절에 따른 loss 변화
    - 하이퍼파라미터에 따른 생성된 가사의 자연스러움
- 모호한 점 :  
    - hidden_size를 수정했을 때에는 뚜렷한 변화가 보였으나 embedding_size 수정시에는 큰 변화가 보이지 않아 어떤 영향을 주었는지 모호하다.

__3. 루브릭 평가 지표를 맞추기 위해 시도한 점__

- embedding_size, hidden_size를 여러번 조절하여 validation loss 2.2로 낮출 수 있었다.

__4. 루브릭 평가 지표를 달성하지 못한 이유__

- 없음

__5. 프로젝트 진행 후 느낀 점 및 다짐__

- 모델 학습을 위한 최적의 하이퍼파라미터의 중요성을 다시 한 번 깨달았다.
- 모델 학습시 정확도는 물론 시간을 줄이기 위한 방법을 알아봐야 겠다.